# Fox Survey Data

## Importing the Data
Tried to import via the API using [QualtricsAPI](https://www.qualtricsapi-pydocs.com/installation.html). I think this may not work cause Jin is the owner of the survey. 

In [1]:
import pandas as pd
import datetime as dt
import os

In [6]:

data_dir = os.path.abspath(os.path.join(os.getcwd(), '../data'))
today = dt.date.today()
date = today.replace(day=today.day-1) #yesterday's data, change as needed

# as long as you remove the time from the datasheet (##.##.csv), you don't need to manually change anything
pre_filename = f'fox-pre_{date.strftime("%B")} {date.strftime("%-d")}, {date.strftime("%Y")}.csv' 
post_filename = f'fox-post_{date.strftime("%B")} {date.strftime("%-d")}, {date.strftime("%Y")}.csv' 

pre = pd.read_csv(os.path.join(data_dir, pre_filename), skiprows=[1,2], encoding='utf-8-sig', parse_dates=[1,2], dayfirst=False)
post = pd.read_csv(os.path.join(data_dir, post_filename), skiprows=[1,2], encoding='utf-8-sig', parse_dates=[1,2], dayfirst=False)


## Demographics

In [50]:
import numpy as np

In [58]:
conditions = {'LEGO':1, 'BIG MOOD':2, 'American Pickers':3, 
              'CNN News':4, 'Continental':5, 'Fox News':6, 
              'NFL':7, 'YouTube TV':8, 'Smartphone A':12,
              'Smartphone B':10, 'Control TV':11, 'Control Smartphone':13}
conditions_swap = {v: k for k, v in conditions.items()}

post['Cond'] = post['condition'].map(conditions_swap)
pre['Cond'] = pre['condition'].map(conditions_swap)

In [56]:
demos = post[['id', 'Cond', 'condition', 'gender', 'age', 'glasses', 'race', 'education', 'income', 'politics']].copy()
demos['F'] = np.where(demos['gender']==2, 1,0)

demos.groupby('Cond').agg({
    'condition': ['size'],
    'age': [min, 'mean', max],
    'F': [sum]
}).round(2)

condition age              F
                        size min   mean max sum
Cond                                           
American Pickers           3  21  21.33  22   3
BIG MOOD                   3  18  21.00  25   2
CNN News                   3  18  20.00  22   3
Continental                3  19  21.33  24   2
Control Smartphone        18  18  24.00  70  13
Control TV                 2  20  26.00  32   2
Fox News                   2  21  25.00  29   1
LEGO                       3  19  22.00  27   2
NFL                        2  21  24.50  28   2

## Rejectors
A lot of rejectors in this group for Fox Sports and Fox News. Some still made their way into the condition they're a rejector of.

In [59]:
rejectors = pre[['sub_id', 'Cond', 'condition', 'rejector_FOX-Prime', 'rejector_FOX-Sports', 'rejector_FOX-News']].copy()

In [63]:
rejectors[['rejector_FOX-Prime', 'rejector_FOX-Sports', 'rejector_FOX-News']].describe()

,rejector_FOX-Prime,rejector_FOX-Sports,rejector_FOX-News
count,39,39,39
unique,2,2,2
top,False,True,True
freq,22,21,31


In [73]:
prime_miss = (rejectors['rejector_FOX-Prime']==True) & (rejectors['Cond']=='LEGO')
sports_miss = (rejectors['rejector_FOX-Sports']==True) & (rejectors['Cond']=='NFL')
news_miss = (rejectors['rejector_FOX-News']==True) & (rejectors['Cond']=='Fox News')

rejectors['miss'] = np.where(prime_miss|sports_miss|news_miss,True,False)
rejectors[(rejectors['miss']==True)]

,sub_id,Cond,condition,rejector_FOX-Prime,rejector_FOX-Sports,rejector_FOX-News,miss
24,sub-025,NFL,7.0,False,True,False,True
26,sub-027,LEGO,1.0,True,False,True,True
31,sub-032,Fox News,6.0,True,True,True,True
32,sub-033,NFL,7.0,True,True,True,True
34,sub-035,LEGO,1.0,True,True,True,True


# Scratch

In [28]:
# how many subjects in each condition
post['Cond'] = post['condition'].map(conditions_swap)
post[['Cond']].groupby('Cond').size()

# average age per group
post.groupby('Cond')[['age']].mean().round(2)

# can we put these together
post.groupby(['Cond', 'gender']).agg({
    'condition': ['size'],
    'age': [min, 'mean', max]
}).round(2)

Cond
American Pickers       3
BIG MOOD               3
CNN News               3
Continental            3
Control Smartphone    18
Control TV             2
Fox News               2
LEGO                   3
NFL                    2
dtype: int64